*IMPORT LIBRARY*

In [16]:
from pathlib import Path
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

*LOAD DATASET*

In [15]:
IMAGE_PATH = Path("./Dataset")
print(f"Checking path: {IMAGE_PATH.resolve()}")  

if not IMAGE_PATH.exists():
    print(f"The directory '{IMAGE_PATH}' does not exist.")
else:
    IMAGE_PATH_LIST = list(IMAGE_PATH.glob("**/*.jpg"))
    print(f'Total Images = {len(IMAGE_PATH_LIST)}')

# print(IMAGE_PATH_LIST)

classes = os.listdir(IMAGE_PATH)
classes = sorted(classes)

print(classes)

for c in classes:
    total_images_class = list(Path(os.path.join(IMAGE_PATH, c)).glob("*/*.jpg"))
    print(f"* {c}: {len(total_images_class)} images")


Checking path: C:\Users\shann\OneDrive\Documents\GitHub\Computer-Vision\Dataset
Total Images = 467
['Acne', 'Bags', 'Redness']
* Acne: 133 images
* Bags: 188 images
* Redness: 146 images


*SKIN SEGMENTATION*

In [4]:
import cv2
import numpy as np

def isskin(image):
    def nothing(x):
        pass

    # Load the image and convert to grayscale
    # gray = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
    # mean, std_dev = cv2.meanStdDev(gray)

    # Create a window with trackbars for parameter adjustment
    cv2.namedWindow('image')

    cv2.createTrackbar('HueMin', 'image', 0, 255, nothing)
    cv2.createTrackbar('HueMax', 'image', 20, 255, nothing)
    cv2.createTrackbar('SatMin', 'image', 48, 255, nothing)
    cv2.createTrackbar('SatMax', 'image', 255, 255, nothing)
    cv2.createTrackbar('ValMin', 'image', 80, 255, nothing)
    cv2.createTrackbar('ValMax', 'image', 255, 255, nothing)

    while True:
        # Get current positions of trackbars
        h_min = cv2.getTrackbarPos('HueMin', 'image')
        h_max = cv2.getTrackbarPos('HueMax', 'image')
        s_min = cv2.getTrackbarPos('SatMin', 'image')
        s_max = cv2.getTrackbarPos('SatMax', 'image')
        v_min = cv2.getTrackbarPos('ValMin', 'image')
        v_max = cv2.getTrackbarPos('ValMax', 'image')

        # Convert the image to HSV color space
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define lower and upper bounds for skin detection
        lower_hsv = np.array([h_min, s_min, v_min])
        upper_hsv = np.array([h_max, s_max, v_max])

        # Create a binary mask where skin regions are white
        skinMask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)

        # Optional: Apply morphological operations to clean up the mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        skinMask = cv2.erode(skinMask, kernel, iterations=2)
        skinMask = cv2.dilate(skinMask, kernel, iterations=2)

        # Blur the mask to smooth the edges
        skinMask = cv2.GaussianBlur(skinMask, (5, 5), 0)

        # Apply the mask to the original image
        skin = cv2.bitwise_and(image, image, mask=skinMask)

        alpha = np.uint8(skinMask > 0) * 255
        result = cv2.merge((skin, alpha))  # Combine BGR with alpha

        # cv2.imshow('Original Image', image)
        # cv2.imshow('Skin Mask', skinMask)
        # cv2.imshow('Skin Detection Result', skin)

        # if cv2.waitKey(1) & 0xFF == 27:
        #     break
        
        # Save the resulting image
        cv2.imwrite("skin_detection_result.png", result)
        
        # Return the processed image
        return result




*PREPROCESSING*

In [18]:
IMG_SIZE = 128

def preprocess_and_extract_features(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to load {image_path}. Skipping.")
        return None
    # skin_img = isskin(img)

    # if skin_img.shape[2] == 4:
    #     skin_img = skin_img[:, :, :3]

    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    hog_features, _ = hog(
        gray,
        orientations=9,  # Number of orientation bins
        pixels_per_cell=(8, 8),  # Size of the cell
        cells_per_block=(2, 2),  # Number of cells per block
        block_norm='L2-Hys',  # Normalization method
        visualize=True,  # Return HOG image for debugging (optional)
        transform_sqrt=True  # Apply power law compression
    )
    return hog_features

*TESTING PREPROCESSING*

In [3]:
def isskin_test(image, h_min=0, h_max=20, s_min=48, s_max=255, v_min=80, v_max=255):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for skin detection
    lower_hsv = np.array([h_min, s_min, v_min])
    upper_hsv = np.array([h_max, s_max, v_max])

    # Create a binary mask where skin regions are white
    skinMask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)

    # Optional: Apply morphological operations to clean up the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skinMask = cv2.erode(skinMask, kernel, iterations=2)
    skinMask = cv2.dilate(skinMask, kernel, iterations=2)

    # Blur the mask to smooth the edges
    skinMask = cv2.GaussianBlur(skinMask, (5, 5), 0)

    # Apply the mask to the original image
    skin = cv2.bitwise_and(image, image, mask=skinMask)

    cv2.imshow('Original Image', image)
    cv2.imshow('Skin Mask', skinMask)
    cv2.imshow('Skin Detection Result', skin)
    cv2.waitKey(0)
    
    # Create an alpha channel
    alpha = np.uint8(skinMask > 0) * 255
    result = cv2.merge((skin, alpha))  # Combine BGR with alpha

    # Save the resulting image
    # cv2.imwrite("skin_detection_result.png", result)
    
    # Return the processed image
    return result

In [6]:
IMG_SIZE = 128

def preprocess_and_extract_features_test(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Failed to load {image_path}. Skipping.")
        return None
    skin_img = isskin_test(img)

    if skin_img.shape[2] == 4:
        skin_img = skin_img[:, :, :3]

    img_resized = cv2.resize(skin_img, (IMG_SIZE, IMG_SIZE))

    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)

    hog_features, _ = hog(
        gray,
        orientations=9,  # Number of orientation bins
        pixels_per_cell=(8, 8),  # Size of the cell
        cells_per_block=(2, 2),  # Number of cells per block
        block_norm='L2-Hys',  # Normalization method
        visualize=True,  # Return HOG image for debugging (optional)
        transform_sqrt=True  # Apply power law compression
    )
    return hog_features

In [12]:
acne = preprocess_and_extract_features_test('./Dataset/Redness/30/front23_jpg.rf.22fa9ac3fc63431c8a20f287e7c711d2.jpg')

*TEST SKIN SEGMENTATION*

In [5]:
IMAGE_PATH_TEST = Path("./Input")
print(f"Checking path: {IMAGE_PATH_TEST.resolve()}")  

if not IMAGE_PATH_TEST.exists():
    print(f"The directory '{IMAGE_PATH_TEST}' does not exist.")
else:
    IMAGE_PATH_TEST_LIST = list(IMAGE_PATH_TEST.glob("**/*.jpg"))
    print(f'Total Images = {len(IMAGE_PATH_TEST_LIST)}')

Checking path: C:\Users\shann\OneDrive\Documents\GitHub\Computer-Vision\Input
Total Images = 5


In [7]:
X = []
y = []

for path in IMAGE_PATH_TEST_LIST:
    print(f"Current path: {path}")
    # print(f"processing: {path}")
    features = preprocess_and_extract_features(path)
    if features is not None:
        X.append(features)
        # y.append(classes.index(path.parent.name))

# Convert features and labels to NumPy arrays for further processing
X = np.array(X)
y = np.array(y)

# Optionally save to file for future use
np.save('features.npy', X)
np.save('labels.npy', y)

X = np.load('features.npy')
y = np.load('labels.npy')
print(f"Loaded features shape: {X.shape}")
print(f"Loaded labels shape: {y.shape}")

Current path: Input\acne.jpg
showed
Current path: Input\eyebags1.jpg
showed
Current path: Input\eyebags2.jpg
showed
Current path: Input\fang.jpg
showed
Current path: Input\red girl.jpg
showed
Loaded features shape: (5, 8100)
Loaded labels shape: (0,)


*LOAD DATASET TO PREPROCESS*

In [19]:
X = []
y = []

for path in IMAGE_PATH_LIST:
    print(f"Current path: {path}, Parent name: {path.parent.parent.name}")
    # print(f"processing: {path}")
    features = preprocess_and_extract_features(path)
    if features is not None:
        X.append(features)
        y.append(classes.index(path.parent.parent.name))

# Convert features and labels to NumPy arrays for further processing
X = np.array(X)
y = np.array(y)

# Optionally save to file for future use
np.save('features.npy', X)
np.save('labels.npy', y)

X = np.load('features.npy')
y = np.load('labels.npy')
print(f"Loaded features shape: {X.shape}")
print(f"Loaded labels shape: {y.shape}")

Current path: Dataset\Acne\0\front.jpg, Parent name: Acne
Current path: Dataset\Acne\0\left_side.jpg, Parent name: Acne
Current path: Dataset\Acne\0\right_side.jpg, Parent name: Acne
Current path: Dataset\Acne\1\front.jpg, Parent name: Acne
Current path: Dataset\Acne\1\left_side.jpg, Parent name: Acne
Current path: Dataset\Acne\1\right_side.jpg, Parent name: Acne
Current path: Dataset\Acne\10\1-2-_jpg.rf.1fcd6581472c267686f8372acf3cf539.jpg, Parent name: Acne
Current path: Dataset\Acne\10\1-3-_jpg.rf.8c6161cbe86b7439c921fc81486e9cd0.jpg, Parent name: Acne
Current path: Dataset\Acne\10\11-1-_jpg.rf.eeef57d0d4d75276fb8128c2aac52d24.jpg, Parent name: Acne
Current path: Dataset\Acne\10\11-3-_jpg.rf.00c4306b0fda6da56c3ac5cf29440468.jpg, Parent name: Acne
Current path: Dataset\Acne\10\114-2-_jpg.rf.ef2cc86b523b140f42c1e27a76114d8f.jpg, Parent name: Acne
Current path: Dataset\Acne\10\114-3-_jpg.rf.4c570f13d93563d217f279cb4b4fa921.jpg, Parent name: Acne
Current path: Dataset\Acne\10\13-2-_jpg.

*CNN*

https://medium.com/@samuelsena/pengenalan-deep-learning-part-7-convolutional-neural-network-cnn-b003b477dc94

In [ ]:
# IMG_SIZE = int(8100 ** 0.5)

In [20]:
IMG_SIZE = 90  # Assuming images are 90x90 pixels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure data is a NumPy array
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Reshape for grayscale images
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# Print shapes
print(f"Training data shape: {X_train.shape}")  # Example: (373, 90, 90, 1)
print(f"Testing data shape: {X_test.shape}")    # Example: (94, 90, 90, 1)


Training data shape: (373, 90, 90, 1)
Testing data shape: (94, 90, 90, 1)


In [1]:
import tensorflow as tf
# from tensorflow import keras
# from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from tensorflow.python.keras.models import Sequential

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

IMG_SIZE = 90  # Assuming images are 90x90
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),  # Grayscale input
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

c:\Users\shann\Python\Python 3.11.8\environments\computer_vision\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 88, 88, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 44, 44, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 61952)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,929,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,930,691 (30.25 MB)

 Trainable params: 7,930,691 (30.25 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [25]:
history = model.fit(
    X_train, y_train, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.2
)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 235ms/step - accuracy: 0.3194 - loss: 2.0232 - val_accuracy: 0.4000 - val_loss: 1.3851
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.5149 - loss: 0.9930 - val_accuracy: 0.5333 - val_loss: 1.0552
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 211ms/step - accuracy: 0.7031 - loss: 0.7622 - val_accuracy: 0.6133 - val_loss: 0.9262
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - accuracy: 0.8320 - loss: 0.6249 - val_accuracy: 0.5600 - val_loss: 0.8729
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 217ms/step - accuracy: 0.9481 - loss: 0.4293 - val_accuracy: 0.5733 - val_loss: 0.8722
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 209ms/step - accuracy: 0.9458 - loss: 0.3242 - val_accuracy: 0.6133 - val_loss: 0.8194
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 217ms/step - accuracy: 0.9754 - loss: 0.2007 - val_accuracy: 0.5733 - val_loss: 0.8459
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step - accuracy: 0.9783 - loss: 0.1560 - val_accuracy: 0.

In [26]:
model.save('cnn_model.h5')

# Load the model
model = tf.keras.models.load_model('cnn_model.h5')

In [27]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

# Display classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_classes, target_names=classes))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
              precision    recall  f1-score   support

        Acne       0.57      0.50      0.53        32
        Bags       0.78      0.64      0.71        28
     Redness       0.58      0.74      0.65        34

    accuracy                           0.63        94
   macro avg       0.65      0.63      0.63        94
weighted avg       0.64      0.63      0.63        94



: 

*TRADITIONAL MODELS*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
print("Training SVM...")
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

svm_y_pred = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
print(f"Accuracy: {svm_accuracy * 100:.2f}%")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, svm_y_pred, target_names=classes))


ValueError: Found input variables with inconsistent numbers of samples: [5, 0]

In [ ]:
print("Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

rf_y_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Accuracy: {rf_accuracy * 100:.2f}%")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, rf_y_pred, target_names=classes))


In [ ]:
print("Training XGB...")
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)

xgb_y_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
print(f"Accuracy: {xgb_accuracy * 100:.2f}%")

# Classification Report
print("Classification Report:")
print(classification_report(y_test, xgb_y_pred, target_names=classes))


In [ ]:
base_models = [
    ('svm', svm_model),
    ('rf', rf_model),
    ('xgb', xgb_model)
]

meta_model1 = LogisticRegression(random_state=42)

stacking_model1 = StackingClassifier(estimators=base_models, final_estimator=meta_model1)

print("Stacking Model with Logistic Regression")
stacking_model1.fit(X_train, y_train)

stacking_y_pred = stacking_model1.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_y_pred)
print(f"Accuracy: {stacking_accuracy * 100:.2f}%")

print("Classification Report:")
print(classification_report(y_test, stacking_y_pred, target_names=classes))


In [ ]:
meta_model2 = RandomForestClassifier(random_state=42)
stacking_model2 = StackingClassifier(estimators=base_models, final_estimator=meta_model2)

print("Stacking Model with Random Forest Classifier")
stacking_model2.fit(X_train, y_train)

stacking_y_pred = stacking_model2.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_y_pred)
print(f"Accuracy: {stacking_accuracy * 100:.2f}%")

print("Classification Report:")
print(classification_report(y_test, stacking_y_pred, target_names=classes))
